<a href="https://colab.research.google.com/github/thaiphi/stock-markets-analytics-zoomcamp/blob/main/Homework_1_Intro_and_Data_Souces_for_Stock_Markets_Analytics_Zoomcamp_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
## Change path to directory of where notebook is located
%cd '/content/drive/MyDrive/Stock Markets Analytics Zoomcamp 2024'

/content/drive/MyDrive/Stock Markets Analytics Zoomcamp 2024


In [ ]:
import duckdb
import pandas as pd

# Connect to DuckDB
con = duckdb.connect(database=':memory:', read_only=False)

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Stock Markets Analytics Zoomcamp 2024/Homework 1: Intro and Data Souces for Stock Markets Analytics Zoomcamp 2024/GDPC1.csv')

# Write the Pandas DataFrame to DuckDB
con.register('gdpc1', df)

# Query the DuckDB table
result = con.execute('SELECT * FROM gdpc1 LIMIT 30')
print(result.fetchdf())

          DATE     GDPC1
0   1947-01-01  2182.681
1   1947-04-01  2176.892
2   1947-07-01  2172.432
3   1947-10-01  2206.452
4   1948-01-01  2239.682
5   1948-04-01  2276.690
6   1948-07-01  2289.770
7   1948-10-01  2292.364
8   1949-01-01  2260.807
9   1949-04-01  2253.128
10  1949-07-01  2276.424
11  1949-10-01  2257.352
12  1950-01-01  2346.104
13  1950-04-01  2417.682
14  1950-07-01  2511.127
15  1950-10-01  2559.214
16  1951-01-01  2593.967
17  1951-04-01  2638.898
18  1951-07-01  2693.259
19  1951-10-01  2699.156
20  1952-01-01  2727.954
21  1952-04-01  2733.800
22  1952-07-01  2753.517
23  1952-10-01  2843.941
24  1953-01-01  2896.811
25  1953-04-01  2919.206
26  1953-07-01  2902.785
27  1953-10-01  2858.845
28  1954-01-01  2845.192
29  1954-04-01  2848.305


https://www.investopedia.com/terms/y/year-over-year.asp

In [ ]:
# prompt: Query the DuckDB table for 2023

result = con.execute('''
SELECT * FROM gdpc1 WHERE
    date >= '2023-01-01' AND date <= '2023-12-31'
    ''')
print(result.fetchdf())



         DATE      GDPC1
0  2023-01-01  22112.329
1  2023-04-01  22225.350
2  2023-07-01  22490.692
3  2023-10-01  22679.255


In [ ]:
# Calculate the YoY growth rate
con.execute('''
    CREATE VIEW gdpc1_yoy AS
    SELECT
        DATE,
        GDPC1,
        LAG(GDPC1, 4) OVER (ORDER BY date) AS value_one_year_ago
    FROM
        gdpc1
''')

# Filter for the year 2023
con.execute('''
    CREATE VIEW gdpc1_yoy_2023 AS
    SELECT
        DATE,
        GDPC1,
        value_one_year_ago,
        ROUND((GDPC1 / value_one_year_ago - 1) * 100, 1) AS yoy_growth_rate
    FROM
        gdpc1_yoy
    WHERE
        date >= '2023-01-01' AND date <= '2023-12-31'
''')

# Calculate the average YoY growth rate for 2023
result = con.execute('SELECT AVG(yoy_growth_rate) FROM gdpc1_yoy_2023')
average_yoy_growth_2023 = result.fetchone()[0]

print(f'Average YoY growth rate in 2023: {average_yoy_growth_2023:.1f}%')

Average YoY growth rate in 2023: 2.5%
